## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch

os.environ["WANDB_SILENT"] = "true"

from owkin.training import train
from owkin.dataset import build_dataset
from owkin.models.mono_models import MLP
from owkin.models.aggregators import SmoothMaxAggregator

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

## Training

In [5]:
num_layers = 5
inside_dim = 512


mono_batch_size = 64
mono_nb_epochs = 2000

val_center = "C_1"
batch_size = 16
nb_epochs = 500

mono_lr = 3e-6
lr = 1e-6


In [6]:
list_val_center = ["C_1", "C_2", "C_5"]
list_normalizer_type = ["None", "mean", "expo_lambda"]
list_wd = [0, 1e-3, 3e-3, 1e-2]

In [8]:
for val_center in list_val_center:
    for normalizer_type in list_normalizer_type:
        X_train, X_train_mean, y_train, X_val, X_val_mean, y_val, X_test, X_test_mean, df_test = build_dataset(normalizer_type=normalizer_type, val_center=val_center)
        for wd in list_wd:
            mono_model = MLP(num_layers=num_layers, inside_dim=inside_dim)
            train(X_train_mean,
                y_train,
                X_val_mean,
                y_val,
                mono_model,
                learning_rate=mono_lr,
                weight_decay=wd,
                val_center=val_center,
                normalizer_type=normalizer_type,
                batch_size=mono_batch_size,
                nb_epochs=mono_nb_epochs,
                use_wandb=True,
                device=device,
            )

            mono_model.load_state_dict(torch.load(mono_model.best_path))
            model = SmoothMaxAggregator(mono_model)
            train(X_train,
                y_train,
                X_val,
                y_val,
                model,
                learning_rate=lr,
                weight_decay=wd,
                val_center=val_center,
                normalizer_type=normalizer_type,
                batch_size=batch_size,
                nb_epochs=nb_epochs,
                use_wandb=True,
                device=device,
                exp_name=f"mono_lr_{mono_lr}"
            )